In [1]:
import pandas as pd
import numpy as np
from datetime import date
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set()
from scipy import stats
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from alpha_vantage.timeseries import TimeSeries
import pandas as pd

In [2]:
#data from the past 15 days 

In [3]:
if __name__ == "__main__":
    #getting data from API
    ts = TimeSeries(key='N1LNTR9K182JZQ3P',output_format='pandas', indexing_type='date')
    data, meta_data = ts.get_intraday(symbol='AAPL',interval='1min', outputsize='full')
    pd.set_option('display.max_rows', None)
    data.head()
    #cleaning the data
    data['open'] =data['1. open'] 
    data['close'] = data['4. close']
    data['high'] = data['2. high']
    data['low'] = data['3. low']
    data1 = data.drop(columns=['1. open','4. close','2. high','3. low','5. volume'])
    data1 = data.reset_index()
    data1.head()
    #reviewing dataTypes
    data1.info()
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7589 entries, 0 to 7588
Data columns (total 10 columns):
date         7589 non-null datetime64[ns]
1. open      7589 non-null float64
2. high      7589 non-null float64
3. low       7589 non-null float64
4. close     7589 non-null float64
5. volume    7589 non-null float64
open         7589 non-null float64
close        7589 non-null float64
high         7589 non-null float64
low          7589 non-null float64
dtypes: datetime64[ns](1), float64(9)
memory usage: 593.0 KB


In [4]:
    #calculating the stock returns
    data1['SR'] = (data1.close - data1.close.shift(1))/data1.close.shift(1)
    data1 = data1.drop([0])
    data1.head()
    #today = date.today()
    #getting data for current day
    yesterDay = data1[(data1['date'].dt.day==20)]
    yesterDay.head()
    #grouping the data by 15 minutes window
    a = yesterDay.groupby(pd.Grouper(freq='15T',key='date')).mean()
    a = a.fillna(0)
    a = a.reset_index()
    a.head()
    #gouring data by yesterday
    xDay = data1[data1['date'].dt.day == 19 ]
    xDay = xDay.drop(columns = ['open','close','high','low'])
    xDay.head()
    #grouping data by 15 minutes window
    b = xDay.groupby(pd.Grouper(freq='15T',key='date')).mean()
    b = b.fillna(0)
    b = b.reset_index()
    b.head()
    #creating regression model 
    X = b[['SR']].astype(float)  # independent variables
    Y = a[['close']].astype(float)  # target variable
    linear_regressor = LinearRegression()  # create object for the class
    linear_regressor.fit(X, Y)  # perform linear regression
    print('Intercept: \n', linear_regressor.intercept_)
    print('Coefficients: \n', linear_regressor.coef_)
    # with statsmodels
    X = sm.add_constant(X) # adding a constant

    model = sm.OLS(Y, X).fit()
    predictions = model.predict(X) 

    print_model = model.summary()
    print(print_model)

Intercept: 
 [461.83420483]
Coefficients: 
 [[-2107.65387739]]
                            OLS Regression Results                            
Dep. Variable:                  close   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.016
Method:                 Least Squares   F-statistic:                  0.002045
Date:                Mon, 24 Aug 2020   Prob (F-statistic):              0.964
Time:                        11:40:06   Log-Likelihood:                -356.05
No. Observations:                  65   AIC:                             716.1
Df Residuals:                      63   BIC:                             720.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------

C:\Users\umers\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [ ]:
#a = yesterDay.groupby(yesterDay['date'].dt.hour).mean()


In [ ]:
#b = xDay.groupby(xDay['date'].dt.hour).mean()
